# Endpoint Changes

## a. Multiple Production Variants

Previous activity: preparing and splitting data

In [ ]:
import numpy as np
import pandas as pd
import boto3
import re
import sagemaker
from sagemaker import get_execution_role

In [ ]:
# Set the file directories
bucket_name = "sagemaker-loan-classification"
prefix = "xgboost"

# Set input and output paths for sagemaker
input_train = sagemaker.TrainingInput(
    s3_data=f"s3://{bucket_name}/{prefix}/train", content_type="csv")
input_test = sagemaker.TrainingInput(
    s3_data=f"s3://{bucket_name}/{prefix}/test", content_type="csv")
output_path = f"s3://{bucket_name}/{prefix}/saved_model"

In [ ]:
# Build the estimator
def xgboost_fit(version, use_spot_instance=False, max_run=3200, max_wait=7200):
    name = f"xgboost-loan-{version}"
    print(f"Running XGBoost version: {}")

    # Set checkpoint if use spot instance
    checkpoint = None
    if use_spot_instances == True:
        checkpoint = f"s3://{bucket_name}/bikerental/checkpoints/{job_name}"
        print(f"Checkout point: {checkpoint}")
        max_wait = None

    # Use the specified version of XGBoost
    container = sagemaker.image_uris.retrieve("xgboost",
                                              sagemaker.Session().boto_region_name,
                                              version=version)

    xgb = sagemaker.estimator.Estimator(
        image_uri=container,
        role=sagemaker.get_execution_role(),
        instance_count=1,
        instance_type="ml.m5.xlarge",
        output_path=output_path,
        sagemaker_session=sagemaker.Session(),
        checkpoint_s3_uri=checkpoint
        use_spot_instance=use_spot_instance,
        max_run=max_run,
        max_wait=max_wait,
        base_job_name=name,
    )

    # Set the hyperparameters
    xgb.set_hyperparameters(
        colsample_bytree=0.478,
        gamma=2.387,
        eta=0.175,
        max_depth=15,
        min_child_weight=7,
        num_round=86,
        subsample=0.80,
        num_class=2,
        objective="multi:softmax",
    )

    # Fit the model
    xgb.fit(
        {
            "train": input_train,
            "validation": input_test
        }
    )

    # Return job name
    return xgb.latest_training_job.name